In [141]:
import pymongo
import pprint
from pymongo import MongoClient
from selenium import webdriver          #Основной элемент
from selenium.webdriver.support.ui import Select   #Поле множественного выбора
from selenium.webdriver.common.keys import Keys    #Клавиши клавиатуры
from selenium.common.exceptions import NoSuchElementException #Отлов ошибки пустоты
from selenium.webdriver.support.ui import WebDriverWait

In [14]:
#Выбор браузера
driver = webdriver.Firefox()

### TASK1

Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить данные о письмах в базу данных (от кого, дата отправки, тема письма, текст письма)

In [15]:
#Открытие страницы
driver.get("https://mail.ru")

In [18]:
#Ввод пароля
assert "Mail" in driver.title
login = driver.find_element_by_id("mailbox:login").send_keys("selenium_test90")
password = driver.find_element_by_id("mailbox:password").send_keys("^YHN^YHN6yhn6yhn")
button = driver.find_element_by_id("mailbox:submit").click()

In [75]:
#Находим поле с полезным содержимым писем. Последнее найденное "а" - не письмо
mails = driver.find_element_by_class_name("dataset__items").find_elements_by_tag_name("a")
len(mails)

4

In [77]:
#Подрубаем MongoDB и задаем коллекцию
client = MongoClient()
db = client.Mailru
emails = db.emails

In [70]:
#Доп.защита от повторений
emails.create_index([('email_id', pymongo.ASCENDING)], unique=True)

'email_id_1'

In [76]:
#собираем данные из писем и кладем их в базу данных
try:
    for i in range(0,(len(mails)-1)):
        sender = mails[i].find_element_by_class_name("ll-crpt").get_attribute("title")
        theme = mails[i].find_element_by_class_name("ll-sj__normal").text
        time = mails[i].find_element_by_class_name("llc__item_date").get_attribute("title")
        snippet = mails[i].find_element_by_class_name("ll-sp__normal").text
        print (f"Время письма: {time}, Отправитель: {sender}, Тема письма: {theme}, Предпросмотр: {snippet}")
        email ={"email_id":mails[i].get_attribute("data-uidl-id"),
               "time":time,
               "sender":sender,
               "theme":theme,
               "snippet":snippet}
        emails.insert_one(email)
except:
    NoSuchElementException
    pymongo.errors.DuplicateKeyError

Время письма: Сегодня, 9:43, Отправитель: Команда Mail.ru <welcome@corp.mail.ru>, Тема письма: Узнайте о супервозможностях Почты Mail.ru, Предпросмотр: Здравствуйте Спасибо, что зарегистрировались в Почте Mail.ru Прочитайте это письмо, чтобы узнать о с
Время письма: Сегодня, 9:43, Отправитель: Команда Почты Mail.ru <welcome@corp.mail.ru>, Тема письма: Mail.ru – больше, чем почта. Познакомьтесь с проектами Mail.ru Group, Предпросмотр: Здравствуйте, selenium! Mail.ru – ваш проводник в интернет. Узнайте больше о проектах Mail.ru Group.
Время письма: Сегодня, 9:43, Отправитель: Команда Почты Mail.ru <welcome@corp.mail.ru>, Тема письма: Как воспользоваться почтой с мобильного?, Предпросмотр: Здравствуйте, selenium! Пользуйтесь Почтой с мобильного Почта всегда рядом, даже если вы далеко от к


### Task 2

Написать программу, которая собирает «Хиты продаж» с сайтов техники mvideo, onlinetrade и складывает данные в БД. Магазины можно выбрать свои. Главный критерий выбора: динамически загружаемые товары

In [189]:
driver.get("https://www.onlinetrade.ru")

In [190]:
assert "Интернет-магазин ОНЛАЙН ТРЕЙД.РУ" in driver.title

In [191]:
#Ищем хиты продаж среди одинаковых заголовков. Находим ноду, содержащую заголовок и ищем ее родителя, в котором уже будут и нужные товары
headers = driver.find_elements_by_class_name("indexGoods__headers")
for i in range(0, len(headers)):
    if headers[i].text == "Хиты продаж":
        header_hit = headers[i]
top_hit = header_hit.find_element_by_xpath("..")

In [192]:
#Все объявления среди хитов продаж
item = top_hit.find_elements_by_class_name("indexGoods__item")

In [193]:
db = client.onlinetrade
top_hits = db.top_hits

In [194]:
for i in range(0,len(item),2):
    name = item[i].find_element_by_class_name("indexGoods__item__descriptionCover").find_element_by_tag_name("a").get_attribute("title").replace("Подробнее о «","").replace("»","")
    link = item[i].find_element_by_class_name("indexGoods__item__descriptionCover").find_element_by_tag_name("a").get_attribute("href")
    price = item[i].find_element_by_class_name("indexGoods__item__price").text 
    print(name, link, price, discount)
    top_hit ={"name":name,
               "link":link,
               "price":price}
    top_hits.insert_one(top_hit)

Фильтр для воды под мойку АКВАФОР DWM-101S Морион, обратный осмос с минерализатором https://www.onlinetrade.ru/catalogue/filtry_dlya_vody_pod_moyku-c240/akvafor/filtr_dlya_vody_pod_moyku_akvafor_dwm_101s_morion_obratnyy_osmos_s_mineralizatorom_4600987006123-140085.html 8 690 ₽ 
Фонарь ЯРКИЙ ЛУЧ V-700 "ВЕЛОФАРА-USB" CREE XM-L2 700лм, IPX4, питание от USB https://www.onlinetrade.ru/catalogue/fary_i_fonari_dlya_velosipedov-c3286/yarkiy_luch/fonar_yarkiy_luch_v_700_velofara_usb_cree_xm_l2_700lm_ipx4_pitanie_ot_usb_4606400106043-1719954.html 2 290 ₽ 
Терка многофункциональная роторная DEKOK UKA-1215 https://www.onlinetrade.ru/catalogue/izmelchiteli_i_rezki-c382/dekok/terka_mnogofunktsionalnaya_rotornaya_dekok_uka_1215-214226.html 1 300 ₽
980 ₽ 
Радионяня Motorola MBP161 Timer белый, B102MBP161TRU https://www.onlinetrade.ru/catalogue/radio_i_videonyani-c518/motorola/radionyanya_motorola_mbp161_timer_belyy_b102mbp161tru-262635.html 2 800 ₽
2 790 ₽ 
Умная Wi-Fi розетка TP-LINK HS100 https://ww